## Import Libraries

In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate
from scipy.spatial import distance
import warnings
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [2]:
case  = 'wat'

if case=='atm':
    theta  = 0
    n      = 1.00029
    c      = 299792458/n
    h      = 25000
    L      = 100
    v      = n * c
    times  = linspace(1e-6,1e-4,300000,endpoint=True)
    
elif case=='wat':
    n      = 1.33
    c      = 299792458/n
    h      = 4.5
    D      = 7.3   #--------------------- diameter of tank
    R      = 7.3/2 #--------------------- radius of tank
    v      = n * c
    times  = linspace(1e-10,1e-7,300000)
    
    c1     = (0, 0, 0)  #------------------------------------ Central    PMT number 1 
    c2     = (1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0) #------ Non-Radial PMT number 2
    c3     = (1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0) #------ Non-Radial PMT number 3
    c4     = (1.85*cos(0     ) , 1.85*sin(0     ), 0) #------ Radial     PMT number 4
    x1,y1  = (0.001,0) #----------------------------------------- Coordinates of particle at top tanker lid      #------PARAM
    theta  = 180   #----------------------------------------- Zenith  varies from 90 to 180                    #------PARAM
    phi    = 0    #----------------------------------------- Azimuth varies from  0 to 360                    #------PARAM
    theta  = deg2rad(theta) #-------------------------------- Convert zenith to radians
    phi    = deg2rad(phi  ) #-------------------------------- Convert azimuth to radians

    x2     = x1 + h * tan(theta) * cos(phi) # ------------------- Calculate bottom x coordinate
    y2     = y1 + h * tan(theta) * sin(phi) #-------------------- Calculate bottom y coordinate
    cp     = (x2,y2,0)
    print('A = {}'.format((x1,y1,h)))
    print('B = {}'.format(cp))

    d1     = distance.euclidean(cp,c1)  #------------------------ Calculate the distances of the point 
    d2     = distance.euclidean(cp,c2)  #------------------------ where the particle hits the bottom of 
    d3     = distance.euclidean(cp,c3)  #------------------------ the tank from the 4 PMTs
    d4     = distance.euclidean(cp,c4)
    theta2    = pi - theta
    del theta
    theta     = theta2
    print('Particle distances from PMTs    : {} m,   {} m,   {} m,   {} m'.format( round(d1,3),round(d2,3),round(d3,3),round(d4,3)))

A = (0.001, 0, 4.5)
B = (0.000999999999999449, 0.0, 0)
Particle distances from PMTs    : 0.001 m,   1.851 m,   1.851 m,   1.849 m


## Computations 

In [26]:
%matplotlib

plotme    = 'phi vs t'
distances = [d1,d2]#,d3,d4]
colors    = ['k','r']#,'b','g']
detectors = [1,2] #,3,4]
labels    = ['Central Detector', "Surrounding Detectors"] #,'Detector 3','Detector 4']

for ds, color, detector,mylabel in zip(distances, colors, detectors, labels):
    
    print("==============================================")
    print("Processing detector %s"%detector)
    
    T, ZP, ZM, BP, BM, PHIP, PHIM = [], [], [], [], [], [], []
    L      = ds
    num_zc = -sqrt(-c**4 * L**2 * cos(theta)**2 + c**2 * L**2 * v**2 * cos(theta)**2) + c**2*L*sin(theta) - L*v**2*sin(theta)
    den    = (c**2-v**2)
    zc     = (num_zc/den)
    zc_act = zc * cos(theta)
    print('z_C : {} m'.format(zc_act))

    for iii,t in enumerate(times):
        z        = h/cos(theta) - v*t
        k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
        t_shower = t
        t_light  = k/c
        t_total  = t_shower + t_light
        tt       = t_total

        num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
        num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
        zp       = ((num1_z - num2_z) / den)
        zm       = ((num1_z + num2_z) / den)
        v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
        v_B      = num2_z
        vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
        vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
        kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
        km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
        alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
        alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
        phip     = 90+theta-rad2deg(alphap)
        phim     = 90+theta-rad2deg(alpham)   
        vtp      = vp * sin(alphap)
        vtm      = vm * sin(alpham)
        omegap   = (vtp / kp)
        omegam   = (vtm / km)
        bp       = abs(omegap / (kp**2))
        bm       = abs(omegam / (km**2))
        
        ZM.append(zm*cos(theta)) 
        ZP.append(zp*cos(theta))
        BM.append(bm)            
        BP.append(bp)
        PHIP.append(phip)
        PHIM.append(phim)
        T.append(tt)
        
    K = sqrt( L**2 + zc**2 - 2*zc*L*cos(pi/2-theta))
    ALPHA   = (arccos( (zc*zc + K*K - L*L) / (2*zc*K) ))
    PHI     = 90+theta-rad2deg(ALPHA)

    ZP, ZM, T, BP, BM, PHIP, PHIM = array(ZP), array(ZM), array(T), array(BP), array(BM), array(PHIP), array(PHIM)
    Tn = T*1e+9 ; TT = (Tn-Tn.min())
    
    cond = ( ((ZP>=0) & (ZP<=h*cos(theta))) | ((ZM>=0) & (ZM<=h*cos(theta))) ) # Improve further if possible
    TT   = TT[cond] ; BM = BM[cond] ; BP = BP[cond] ; PHIP = PHIP[cond] ; PHIM = PHIM[cond] ; ZP=ZP[cond] ; ZM=ZM[cond]
    
    if detector==1:
        den_bright = BP[-1]
    BM = BM / den_bright
    BP = BP / den_bright
    
    if plotme == 'b vs phi': # Use times  = linspace(1e-12,1e-7,2000000)
        if detector==1:
            plt.plot(PHIP, BP, c=color, ls='-', label=mylabel)
        else:
            plt.plot(PHIP, BP, c=color, ls='-', label=mylabel)
            plt.plot(PHIM, BM, c=color, ls='--')
        plt.axhline(1,c='k',ls=':')
        plt.axvline(PHI,c='k',ls=':')
        plt.text(PHI-3, 2, r'$\theta_C$ = {}'.format(round(PHI,2))     , fontsize=18)
        plt.xlabel(r'angular locations $\phi_{pm}\;(in\;degrees)$', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        plt.xlim(0,91)

    elif plotme == 'b vs t': # Use times  = linspace(1e-10,1e-7,300000)
        if detector==1:
            plt.plot(TT, BP, c=color, ls='-' , label=mylabel)
        else:
            plt.plot(TT, BP, c=color, ls='-' , label=mylabel)
            plt.plot(TT, BM, c=color, ls='--')
        
        plt.axhline(1, c='k', ls=':')
        plt.axvline(0, c='k', ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        
    elif plotme == 'phi vs t':
        if detector==1:
            plt.plot(TT, PHIP, c=color, ls='-' , label=mylabel)
        else:
            plt.plot(TT, PHIP, c=color, ls='-' , label=mylabel)
            plt.plot(TT, PHIM, c=color, ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.axhline(PHI, c='k', ls='--')
        plt.text(2, PHI+1.5, r'$\theta_C = 48.75^{\circ}$' , fontsize=15) #%(PHI)
        plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
        plt.ylim(-0.1,91)
            
    print()
    del BM, BP, TT, PHIM, PHIP, ZM, ZP, T
    
plt.legend(loc='lower right', prop={'size': 14})
plt.show()

Using matplotlib backend: Qt5Agg
Processing detector 1
z_C : 0.001140420641900159 m

Processing detector 2
z_C : 2.1103486289412796 m



In [ ]:
def entry_brightness(h,c,v,theta=0,L=0):
    z        = h/cos(theta)
    k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
    t_shower = 0
    t_light  = k/c
    t_total  = t_shower + t_light
    tt       = t_total
    den      = (c**2-v**2)
    num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
    v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
    v_B      = num2_z
    vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
    vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
    kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
    km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
    alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
    alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
    vtp      = vp * sin(alphap)
    vtm      = vm * sin(alpham)
    omegap   = (vtp / kp)
    omegam   = (vtm / km)
    bp       = abs(omegap / (kp**2))
    bm       = abs(omegam / (km**2))
    return bm,bp

entry_brightness(h=h,c=c,v=v,theta=0,L=0)